In [1]:
# pip install -r compute/client_library/requirements.txt


# https://cloud.google.com/docs/authentication/application-default-credentials
# Put it near notebook file

In [2]:
# Full process described in https://cloud.google.com/compute/docs/instances/moving-instance-across-zones#python

In [3]:
import os 
from pathlib import Path
import time

start_time = time.time()

credentials_path = Path(os.getcwd()) / 'application_default_credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(credentials_path)

In [4]:
from compute.client_library.snippets.instances.get import get_instance

current_zone = "us-west1-b"
# current_zone = "europe-southwest1-a"
current_instance_name = "vm"
project_id = "test-smart-scheduler"

instance = get_instance(
    project_id=project_id, zone=current_zone, instance_name=current_instance_name
)

In [16]:
machine_type = instance.machine_type.split("/")[-1]
disk = instance.disks[0]
disk.disk_size_gb, disk.device_name, machine_type

(25, 'persistent-disk-0', 'e2-medium')

In [17]:
from compute.client_library.snippets.disks.autodelete_change import set_disk_autodelete

set_disk_autodelete(
    project_id=project_id,
    zone=current_zone,
    instance_name=current_instance_name,
    disk_name=disk.device_name,
    autodelete=False,
)

In [18]:
from compute.client_library.snippets.snapshots.create import create_snapshot

snapshot_name = f"{disk.device_name}-snapshot"

snapshot = create_snapshot(
    project_id=project_id,
    disk_name=disk.device_name,
    snapshot_name=snapshot_name,
    zone=current_zone,
)

In [19]:
snapshot

architecture: "X86_64"
creation_size_bytes: 21844277760
creation_timestamp: "2023-06-15T00:28:19.231-07:00"
disk_size_gb: 25
download_bytes: 21844475298
id: 1138803013925143423
kind: "compute#snapshot"
label_fingerprint: "42WmSpB8rSM="
license_codes: 591260547102547226
licenses: "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-minimal-2204-lts"
name: "persistent-disk-0-snapshot"
self_link: "https://www.googleapis.com/compute/v1/projects/test-smart-scheduler/global/snapshots/persistent-disk-0-snapshot"
source_disk: "https://www.googleapis.com/compute/v1/projects/test-smart-scheduler/zones/europe-southwest1-a/disks/persistent-disk-0"
source_disk_id: "5954479496213504791"
status: "READY"
storage_bytes: 21844277760
storage_bytes_status: "UP_TO_DATE"
storage_locations: "eu"

In [20]:
from compute.client_library.snippets.instances.delete import delete_instance

delete_instance(
    project_id=project_id, zone=current_zone, machine_name=current_instance_name
)

Deleting vm from europe-southwest1-a...
Instance vm deleted.


In [21]:
from compute.client_library.snippets.disks.delete import delete_disk

delete_disk(project_id=project_id, zone=current_zone, disk_name=disk.device_name)

In [22]:
from compute.client_library.snippets.disks.create_from_snapshot import (
    create_disk_from_snapshot,
)

# new_zone = "europe-southwest1-a"
new_zone = "us-west1-b"


new_disk = create_disk_from_snapshot(
    project_id=project_id,
    zone=new_zone,
    disk_name=disk.device_name,
    disk_size_gb=disk.disk_size_gb,
    disk_type=f"zones/{new_zone}/diskTypes/pd-standard",
    snapshot_link=snapshot.self_link,
)

In [23]:
from compute.client_library.snippets.instances.create_start_instance.create_with_existing_disks import (
    create_instance,
    get_disk,
    compute_v1,
)


new_instance_name = "vm"


def create_with_existing_disks(
    project_id: str,
    zone: str,
    instance_name: str,
    disk_names: list[str],
    machine_type: str = "n1-standard-1",
) -> compute_v1.Instance:
    """
    Create a new VM instance using selected disks. The first disk in disk_names will
    be used as boot disk.

    Args:
        project_id: project ID or project number of the Cloud project you want to use.
        zone: name of the zone to create the instance in. For example: "us-west3-b"
        instance_name: name of the new virtual machine (VM) instance.
        disk_names: list of disk names to be attached to the new virtual machine.
            First disk in this list will be used as the boot device.


    Returns:
        Instance object.
    """
    assert len(disk_names) >= 1
    disks = [get_disk(project_id, zone, disk_name) for disk_name in disk_names]
    attached_disks = []
    for disk in disks:
        adisk = compute_v1.AttachedDisk()
        adisk.source = disk.self_link
        attached_disks.append(adisk)
    attached_disks[0].boot = True
    instance = create_instance(
        project_id,
        zone,
        instance_name,
        attached_disks,
        machine_type=machine_type,
        external_access=True,
    )
    return instance


new_instance = create_with_existing_disks(
    project_id=project_id,
    zone=new_zone,
    instance_name=new_instance_name,
    disk_names=[new_disk.name],
    machine_type=machine_type,
)

Creating the vm instance in us-west1-b...
Instance vm created.


In [24]:
new_instance.network_interfaces[0].access_configs[0].nat_i_p

'34.105.9.76'

In [25]:
from compute.client_library.snippets.snapshots.delete import delete_snapshot
delete_snapshot(project_id=project_id, snapshot_name=snapshot_name)

In [26]:
end_time = time.time()
end_time - start_time

394.0632379055023